In [1]:
from google.colab import drive
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd
os.chdir("drive/MyDrive/Kolektif Öğrenme/project")
!pwd

/content
/content/drive/MyDrive/Kolektif Öğrenme/project


In [3]:
import cv2
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RepeatedKFold
import math
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras.layers import GlobalAveragePooling2D, Dense, MaxPooling2D, Flatten, Dropout, Input
from keras.models import Model
from itertools import combinations as comb
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
import xgboost as xgb

import warnings
warnings.filterwarnings("ignore")

In [4]:
IMAGE_SIZE = 128

In [30]:
kirmizi_folder = "./Pistachio_Image_Dataset/Kirmizi_Pistachio"
siirt_folder = "./Pistachio_Image_Dataset/Siirt_Pistachio"

In [6]:
def resize_image(image, image_size):
    return cv2.resize(image.copy(), image_size, interpolation=cv2.INTER_AREA)

In [7]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(resize_image(img, (IMAGE_SIZE, IMAGE_SIZE)) / 255.)
    return np.asarray(images)

In [8]:
def get_inception():
    inception_model = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
    )
    x = inception_model.output
    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(128, activation="relu")(x)
    x = Dropout(0.5)(x)
    output = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inception_model.input, outputs=output)
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["acc"])

    return inception_model, model

In [9]:
def incept_rf(base_model, modely, train_imgs, test_imgs):
    X_train_features = []
    X_valid_features = []

    feature_network = Model(base_model.input, modely.get_layer(list(filter(lambda x: "flatten" in x.name,  model.layers))[0].name).output)
    X_train_features = feature_network.predict(train_imgs)
    X_valid_features = feature_network.predict(test_imgs)
    return X_train_features, X_valid_features

In [10]:
def generate_imp_space(X_train, y_train, X_test, imp_feature_size, foz):
    imp_train_data = X_train.values
    imp_test_data = X_test.values
    y_train = pd.DataFrame(y_train)
    d = len(X_train.columns)

    for i in range(0, imp_feature_size*foz):
        Xindis = np.random.permutation(d)
        for j in range(0, d-(foz-1), foz):
            sX = np.random.permutation(1)
            s1 = sX[0]

            s1data = X_train[X_train.index.isin(y_train[y_train == str(s1)].index)]
            s2data = X_train[~X_train.index.isin(y_train[y_train == str(s1)].index)]
            s1data = s1data.iloc[:, Xindis[j:j+(foz)]]
            s2data = s2data.iloc[:, Xindis[j:j+(foz)]]

            s1label = np.ones((s1data.values.shape[0], 1), dtype=int)
            s2label = -1*np.ones((s2data.values.shape[0], 1), dtype=int)
            Wdata = np.concatenate((s1data, s2data))

            Wdata = x2fx(Wdata)
            Wlabel = np.concatenate((s1label, s2label))
            W = np.matmul(np.matmul(np.linalg.pinv(
                np.matmul(Wdata.T, Wdata)), Wdata.T), Wlabel)

            WW = x2fx(X_train.iloc[:, Xindis[j:j+(foz)]].values)
            imp_train_data = np.concatenate(
                (imp_train_data, np.matmul(WW, W)), axis=1)

            TT = x2fx(X_test.iloc[:, Xindis[j:j+(foz)]].values)
            imp_test_data = np.concatenate(
                (imp_test_data, np.matmul(TT, W)), axis=1)

    return imp_train_data, imp_test_data


In [11]:
def x2fx(x, model="linear"):
    linear = np.c_[np.ones(x.shape[0]), x]
    if model == "linear":
        return linear
    if model == "purequadratic":
        return np.c_[linear, x**2]
    interaction = np.hstack([x[:, i]*x[:, j]
                            for i, j in comb(range(x.shape[1]), 2)]).T
    if model == "interaction":
        return np.c_[linear, interaction]
    if model == "quadratic":
        return np.c_[linear, interaction, x**2]

In [31]:
kirmizi = load_images_from_folder(kirmizi_folder)
siirt = load_images_from_folder(siirt_folder)

X = np.concatenate([kirmizi, siirt])
y = np.concatenate([np.ones(kirmizi.shape[0]), np.zeros(siirt.shape[0])])

del(kirmizi)
del(siirt)

In [32]:
cv = RepeatedKFold(n_splits=2, n_repeats=5, random_state=42)
foz = 4
imp_feature_size = 1
n_estimators = 3
imp_rfc_acc = []
rfc_acc = []

for train_index, test_index in cv.split(X, y):
    estimators_imp = []
    estimators = []
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

    inception_model, model = get_inception()
    history = model.fit(X_train, y_train, epochs=5, batch_size=64)

    X_train_features, X_valid_features = incept_rf(
        inception_model, model, X_train, X_test)
    X_train_features = pd.DataFrame(X_train_features)
    X_valid_features = pd.DataFrame(X_valid_features)
    d = len(X_train_features.columns)

    for i in range(n_estimators):
        imp_tr, imp_ts = generate_imp_space(
            X_train_features, y_train, X_valid_features, imp_feature_size, foz)
        imp_d = imp_tr.shape[1]

        imp_sel_d = 2 * round(math.log2(imp_d))
        sel_d = 2*round(math.log2(d))

        imp_rfc = RandomForestClassifier(
            max_features=imp_sel_d, n_estimators=50, random_state=42)
        estimators_imp.append(("imp_rfc"+str(i), imp_rfc))
        imp_rfc.fit(imp_tr, y_train)

        rfc = RandomForestClassifier(
            max_features=sel_d, n_estimators=50, random_state=42)
        estimators.append(("rfc"+str(i), rfc))
        rfc.fit(X_train_features, y_train)
    
    voting_imp = VotingClassifier(estimators=estimators_imp)
    voting_imp.fit(imp_tr, y_train)

    voting_rfc = VotingClassifier(estimators=estimators)
    voting_rfc.fit(X_train_features, y_train)

    imp_rfc_acc.append(voting_imp.score(imp_ts, y_test))
    rfc_acc.append(voting_rfc.score(X_valid_features, y_test))


87910968/87910968 [==============================] - 5s 0us/step
Epoch 1/5
17/17 [==============================] - 24s 316ms/step - loss: 0.6339 - acc: 0.6927
Epoch 2/5
17/17 [==============================] - 3s 159ms/step - loss: 0.2109 - acc: 0.9078
Epoch 3/5
17/17 [==============================] - 3s 154ms/step - loss: 0.1317 - acc: 0.9469
Epoch 4/5
17/17 [==============================] - 3s 152ms/step - loss: 0.1641 - acc: 0.9367
Epoch 5/5
34/34 [==============================] - 1s 24ms/step
Epoch 1/5
17/17 [==============================] - 10s 155ms/step - loss: 0.4406 - acc: 0.8054
Epoch 2/5
17/17 [==============================] - 3s 156ms/step - loss: 0.2182 - acc: 0.9125
Epoch 3/5
17/17 [==============================] - 3s 156ms/step - loss: 0.1407 - acc: 0.9600
Epoch 4/5
17/17 [==============================] - 3s 158ms/step - loss: 0.1043 - acc: 0.9618
Epoch 5/5
34/34 [==============================] - 1s 22ms/step
Epoch 1/5
17/17 [==============================] - 10

In [33]:
cv = RepeatedKFold(n_splits=2, n_repeats=5, random_state=42)
foz = 4
imp_feature_size = 1
n_estimators = 3
imp_xgbc_acc = []
xgbc_acc = []
y_pred_imp_rfc = []
for train_index, test_index in cv.split(X, y):
    estimators_imp = []
    estimators = []
    X_train = X[train_index]
    X_test = X[test_index]
    y_train = y[train_index]
    y_test = y[test_index]

    inception_model, model = get_inception()
    history = model.fit(X_train, y_train, epochs=5, batch_size=64)

    X_train_features, X_valid_features = incept_rf(
        inception_model, model, X_train, X_test)
    X_train_features = pd.DataFrame(X_train_features)
    X_valid_features = pd.DataFrame(X_valid_features)
    d = len(X_train_features.columns)

    for i in range(n_estimators):
        imp_tr, imp_ts = generate_imp_space(
            X_train_features, y_train, X_valid_features, imp_feature_size, foz)
        imp_d = imp_tr.shape[1]

        imp_sel_d = 2 * round(math.log2(imp_d))
        sel_d = 2*round(math.log2(d))

        imp_xgbc = xgb.XGBClassifier(
            max_features=imp_sel_d, n_estimators=50, random_state=42)
        estimators_imp.append(("imp_xgbc"+str(i), imp_rfc))
        imp_xgbc.fit(imp_tr, y_train)

        xgbc = xgb.XGBClassifier(
            max_features=sel_d, n_estimators=50, random_state=42)
        estimators.append(("xgbc"+str(i), rfc))
        xgbc.fit(X_train_features, y_train)
    
    voting_imp = VotingClassifier(estimators=estimators_imp)
    voting_imp.fit(imp_tr, y_train)

    voting_xgbc = VotingClassifier(estimators=estimators)
    voting_xgbc.fit(X_train_features, y_train)

    imp_xgbc_acc.append(voting_imp.score(imp_ts, y_test))
    xgbc_acc.append(voting_xgbc.score(X_valid_features, y_test))

Epoch 1/5
17/17 [==============================] - 10s 162ms/step - loss: 0.5657 - acc: 0.7709
Epoch 2/5
17/17 [==============================] - 3s 163ms/step - loss: 0.2753 - acc: 0.8920
Epoch 3/5
17/17 [==============================] - 3s 163ms/step - loss: 0.1950 - acc: 0.9190
Epoch 4/5
17/17 [==============================] - 3s 164ms/step - loss: 0.1057 - acc: 0.9553
Epoch 5/5
34/34 [==============================] - 1s 23ms/step
Epoch 1/5
17/17 [==============================] - 10s 161ms/step - loss: 0.5265 - acc: 0.7495
Epoch 2/5
17/17 [==============================] - 3s 163ms/step - loss: 0.2003 - acc: 0.9181
Epoch 3/5
17/17 [==============================] - 3s 164ms/step - loss: 0.1373 - acc: 0.9525
Epoch 4/5
17/17 [==============================] - 3s 164ms/step - loss: 0.0819 - acc: 0.9786
Epoch 5/5
34/34 [==============================] - 1s 23ms/step
Epoch 1/5
17/17 [==============================] - 10s 163ms/step - loss: 0.5882 - acc: 0.7169
Epoch 2/5
17/17 [======

In [34]:
print(imp_xgbc_acc)

[0.8985102420856611, 0.946927374301676, 0.9618249534450651, 0.9422718808193669, 0.9385474860335196, 0.9422718808193669, 0.9292364990689013, 0.9506517690875232, 0.9543761638733705, 0.9357541899441341]


In [36]:
print(xgbc_acc)

[0.8947858472998138, 0.9497206703910615, 0.9646182495344506, 0.9404096834264432, 0.9385474860335196, 0.9478584729981379, 0.931098696461825, 0.9497206703910615, 0.952513966480447, 0.9376163873370578]


In [38]:
print(imp_rfc_acc)

[0.9143389199255121, 0.9506517690875232, 0.957169459962756, 0.9068901303538175, 0.9394785847299814, 0.9394785847299814, 0.9003724394785847, 0.952513966480447, 0.9162011173184358, 0.9152700186219739]


In [39]:
print(rfc_acc)

[0.9162011173184358, 0.9543761638733705, 0.9581005586592178, 0.9124767225325885, 0.9441340782122905, 0.9385474860335196, 0.9022346368715084, 0.9534450651769087, 0.9245810055865922, 0.9236499068901304]
